<a href="https://colab.research.google.com/github/scomarae/SkillBox/blob/main/%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81%D0%BE_%D1%81%D1%82%D1%80%D0%BE%D0%BA%D0%B0%D0%BC%D0%B8_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Подготовка данных для задания

In [4]:
!pip install pymorphy2

In [5]:
import re

import pandas as pd
import pymorphy2

text_df = pd.read_csv("/content/drive/MyDrive/SkillBox/content_description.csv", sep='\t')
text_df.head()

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)

# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower)
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
29,0,и,7
37,0,который,5
111,0,шерлок,4
77,0,с,4
84,0,сериал,4
75,0,риколетти,3
31,0,история,2
21,0,для,2
49,0,о,2


# 12.3  Домашняя работа

12.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процент документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [6]:
#считаю кол-во документов
docs_count = len(tokens_df['doc_id'].unique())

# считаю в скольких документах есть слово
doc_frequency = word_count_df.groupby('word')['doc_id'].count().sort_values(ascending=False).reset_index()
doc_frequency.rename(columns={'doc_id': 'counts'}, inplace = True)

#добавляю столбец с частотой
doc_frequency['freq'] = (doc_frequency['counts']/docs_count)*100
doc_frequency.head(10)

,word,counts,freq
0,и,8,100.0
1,в,8,100.0
2,на,7,87.5
3,с,7,87.5
4,он,6,75.0
5,свой,6,75.0
6,что,6,75.0
7,онлайн,6,75.0
8,не,6,75.0
9,год,5,62.5


12.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [7]:
raw_text = text_df.description.values[3]

reg_exp=r'[A-Я]\w+ [A-Я]\w+'
reg_comp = re.compile(reg_exp)

reg_comp.findall(raw_text)

# сюда попадают два лишних значения, но думаю так и должно быть? опсиание регулярки подразумевает

['Стивена Фрая', 'Джоном Дженксом', 'Теда Уоллеса', 'Теперь Тед', 'Вскоре Тед']

10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`, сохранить в переменнуб `genres_df`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [17]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'],
    'триллер': ['ужас', 'зловещий', 'нерв']
}

nested_g = [[(i, j) for j in genre_dict[i]] for i in genre_dict]
flatten_g = list(itertools.chain(*nested_g))
genres_df = pd.DataFrame(flatten_g, columns=['genre', 'word'])

genres_df.merge(word_count_df, on='word', how='inner').groupby('doc_id')['genre'].apply(lambda x: ', '.join(x)).reset_index()
# как тут проверку добавить, чтобы уже имеющиеся слова не входили в строку?

,doc_id,genre
0,0,"детектив, детектив, детектив, триллер"
1,1,сказка
2,2,"мелодрама, мелодрама"
3,3,комедия
4,4,"комедия, комедия"
5,6,"триллер, триллер, триллер"
